### Recommendation Engine

#### Importation of libraries and connection of Pyrhon with AdventureworksDW Database

In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import matplotlib.pyplot as plt
import  matplotlib 
import pyodbc
from scipy.sparse.linalg import spsolve

connection = pyodbc.connect('Driver={SQL Server};Server=DESKTOP-5QU9IGN;Database=AdventureWorksDW2017')

cursor = connection.cursor() # python is connected successfully with the AdventureWorksdatabase
cursor


#### Import the SQL Query

In [2]:
Retail = pd.read_sql_query("""

SELECT
A.[SalesOrderNumber] AS TransactionID,
A.[OrderQuantity] AS Quantity,
A.[UnitPrice],
A.[OrderDate],
C.[ProductKey] AS ProductID,
C.[EnglishProductName] AS ProductName,
B.[CustomerKey]AS CustomerID,
D.[SalesTerritoryCountry] AS Country

FROM [dbo].[FactInternetSales] A
LEFT JOIN [dbo].[DimCustomer] B
ON A.CustomerKey = B.CustomerKey
LEFT JOIN[dbo].[DimProduct] C
ON A.ProductKey = C.ProductKey
LEFT JOIN [dbo].[DimSalesTerritory]D
ON A.SalesTerritoryKey =D.SalesTerritoryKey
ORDER BY A.[SalesOrderNumber]
""",connection)

In [3]:
Retail.head()

,TransactionID,Quantity,UnitPrice,OrderDate,ProductID,ProductName,CustomerID,Country
0,SO43697,1,3578.2700,2010-12-29,310,"Road-150 Red, 62",21768,Canada
1,SO43698,1,3399.9900,2010-12-29,346,"Mountain-100 Silver, 44",28389,France
2,SO43699,1,3399.9900,2010-12-29,346,"Mountain-100 Silver, 44",25863,United States
3,SO43700,1,699.0982,2010-12-29,336,"Road-650 Black, 62",14501,United States
4,SO43701,1,3399.9900,2010-12-29,346,"Mountain-100 Silver, 44",11003,Australia


### Data Exploration

#### Checking for missing Data

In [4]:
Retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60398 entries, 0 to 60397
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   TransactionID  60398 non-null  object        
 1   Quantity       60398 non-null  int64         
 2   UnitPrice      60398 non-null  float64       
 3   OrderDate      60398 non-null  datetime64[ns]
 4   ProductID      60398 non-null  int64         
 5   ProductName    60398 non-null  object        
 6   CustomerID     60398 non-null  int64         
 7   Country        60398 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 3.7+ MB


#### Before we make any sort of ratings matrix, it would be nice to have a lookup table that keeps track of each item ID along with a description of that item

In [5]:
item_lookup = Retail[['ProductID', 'ProductName']].drop_duplicates() # Only get unique item/names pairs
item_lookup['ProductID'] = item_lookup.ProductID.astype(str) # Encode as strings for future lookup ease

In [6]:
item_lookup.head()

,ProductID,ProductName
0,310,"Road-150 Red, 62"
1,346,"Mountain-100 Silver, 44"
3,336,"Road-650 Black, 62"
5,311,"Road-150 Red, 44"
7,351,"Mountain-100 Black, 48"


### Matrix Rating

#### We need to:

#### Group purchase quantities together by productID and customerID Change any sums that equal zero to one (this can happen if items were returned, but we want to indicate that the user actually purchased the item instead of assuming no interaction between the user and the item ever took place) Only include customers with a positive purchase total to eliminate possible errors Set up our sparse ratings matrix

In [7]:
Retail['CustomerID'] = Retail.CustomerID.astype(int) # Convert to int for customer ID
Retail = Retail[['ProductID', 'Quantity', 'CustomerID']] # Get rid of unnecessary info
grouped_Retail = Retail.groupby(['CustomerID', 'ProductID']).sum().reset_index() # Group together
grouped_Retail.Quantity.loc[grouped_Retail.Quantity == 0] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
grouped_purchased = grouped_Retail.query('Quantity > 0') # Only get customers where purchase totals were positive


In [8]:
grouped_purchased.head()

,CustomerID,ProductID,Quantity
0,11000,214,1
1,11000,344,1
2,11000,353,1
3,11000,485,1
4,11000,488,1


In [9]:
grouped_purchased['ProductID']=grouped_purchased['ProductID'].astype(str)# Convert to string for ProductID

#### Our last step is to create the sparse ratings matrix of users and items utilizing the code below:

In [10]:
customers = list(np.sort(grouped_purchased.CustomerID.unique())) # Get our unique customers
products = list(grouped_purchased.ProductID.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Quantity) # All of our purchases

In [11]:
len(customers)

18484

In [12]:
len(products)

158

In [13]:
len(quantity)

59051

#### Creating Sparse Rating Matrix

In [14]:
from pandas.api.types import CategoricalDtype
#get the associated row indices
rows= grouped_purchased.CustomerID.astype(CategoricalDtype(categories=customers)).cat.codes
#get the associated column indices
cols= grouped_purchased.ProductID.astype(CategoricalDtype(categories=products)).cat.codes
#compress sparse row matrix
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [15]:
purchases_sparse
#a sparse matrix is not a pandas dataframe, but sparse matrices are efficient for row slicing and fast matrix vector products

<18484x158 sparse matrix of type '<class 'numpy.intc'>'
	with 59051 stored elements in Compressed Sparse Row format>

#### We have 18484 customers with 158 items. For these user/item interactions, 59051 of these items had a purchase. In terms of sparsity of the matrix, that makes:

In [16]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

97.97803231806365

#### 97.97% of the interaction matrix is sparse. For collaborative filtering to work, the maximum sparsity you could get away with would probably be about 99.5% or so. We are well below this, so we should be able to get decent results.

### Creating a Training and Validation Set

#### Typically in Machine Learning applications, we need to test whether the model we just trained is any good on new data it hasn’t yet seen before from the training phase. We do this by creating a test set completely separate from the training set. Usually this is fairly simple: just take a random sample of the training example rows in our feature matrix and separate it away from the training set.

#### With collaborative filtering, that’s not going to work because you need all of the user/item interactions to find the proper matrix factorization. A better method is to hide a certain percentage of the user/item interactions from the model during the training phase chosen at random. Then, check during the test phase how many of the items that were recommended the user actually ended up purchasing in the end. Ideally, you would ultimately test your recommendations with some kind of A/B test or utilizing data from a time series where all data prior to a certain point in time is used for training while data after a certain period of time is used for testing.

#### Our test set is an exact copy of our original data. The training set, however, will mask a random percentage of user/item interactions and act as if the user never purchased the item (making it a sparse entry with a zero). We then check in the test set which items were recommended to the user that they ended up actually purchasing. If the users frequently ended up purchasing the items most recommended to them by the system, we can conclude the system seems to be working.

#### Now that we have a plan on how to separate our training and testing sets, let’s create a function that can do this for us. We will also import the random library and set a seed so that you will see the same results as I did.

In [17]:
import random 

In [18]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

#This will return our training set, a test set that has been binarized to 0/1 for purchased/not purchased, 
#and a list of which users had at least one item masked. We will test the performance of the recommender system on these users only. 
#I am masking 20% of the user/item interactions for this project.

In [19]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

### Now that we have our train/test split, it is time to implement the alternating least squares algorithm

#### Implementing ALS for Implicit Feedback


In [20]:
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts. 
    
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
    return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                         # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

In [21]:
user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1,
                                            rank_size = 20)

In [22]:
#We can investigate ratings for a particular user by taking the dot product between the user and item vectors (U and V)./
#Let’s look at our first user.

user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

array([5.24299219e-02, 9.85570586e-05, 1.25361284e-02, 3.89815739e-02,
       7.83943702e-03])

#### This is a sample of the first five items out of the 158 in our stock. The first user in our matrix has the second item with the greatest recommendation out of the first five items. However, notice we only did one iteration because our algorithm was so slow! You should iterate at least ten times according to the authors so that U and V converge.This is done using matrix factorization

#### Speeding up ALS by importing the library for matrix factorization

In [23]:
import implicit

In [24]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


  0%|          | 0/50 [00:00<?, ?it/s]

### Evaluating the Recommender System

#### Remember that our training set had 20% of the purchases masked? This will allow us to evaluate the performance of our recommender system. Essentially, we need to see if the order of recommendations given for each user matches the items they ended up purchasing. A commonly used metric for this kind of problem is the area under the Receiver Operating Characteristic (or ROC) curve. A greater area under the curve means we are recommending items that end up being purchased near the top of the list of recommended items.

In [25]:
from sklearn import metrics

In [26]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   

#### Now, utilize this helper function inside of a second function that will calculate the AUC for each user in our training set that had at least one item masked. It should also calculate AUC for the most popular items for our users to compare.

In [27]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

#### We can now use this function to see how our recommender system is doing. To use this function, we will need to transform our output from the ALS function to csr_matrix format and transpose the item vectors. The original pure Python version output the user and item vectors into the correct format already.

In [28]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.773, 0.836)

#### With an AUC OF 0.77,  we need to find a way of retrieving the items already purchased by a user in the training set. Initially, we will create an array of our customers and items we made earlier.

In [29]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [30]:
#Now, we can create a function that will return a list of the item descriptions from our earlier created item lookup table.

def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product names available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the product IDs for our purchased items
    return item_lookup.loc[item_lookup.ProductID.isin(prod_codes)]
    

In [31]:
#We need to look these up by a customer’s ID. Looking at the list of customers:
customers_arr[:5]

array([11000, 11001, 11002, 11003, 11004], dtype=int64)

In [32]:
#we can see that the first customer listed has an ID of 11000. Let’s examine their purchases from the training set.
get_items_purchased(11000, product_train, customers_arr, products_arr, item_lookup)

,ProductID,ProductName
5433,353,"Mountain-200 Silver, 38"
5440,214,"Sport-100 Helmet, Red"
5457,541,Touring Tire
5458,530,Touring Tire Tube
5459,573,"Touring-1000 Blue, 46"
5475,485,Fender Set - Mountain


#### We can see that the customer purchased 6 items from the store. What items does the recommender system say this customer should purchase? We need to create another function that does this. Let’s also import the MinMaxScaler from scikit-learn to help with this.

In [33]:
from sklearn.preprocessing import MinMaxScaler

In [34]:
#Now, we can create a function that will return a list of the items recommended from our earlier created item lookup table.

def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.ProductName.loc[item_lookup.ProductID == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    ProductName = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'ProductID': codes, 'ProductName': ProductName}) # Create a dataframe 
    return final_frame[['ProductID', 'ProductName']] # Switch order of columns around

#### Essentially, this will retrieve the N highest ranking dot products between our user and item vectors for a particular user. Items already purchased are not recommended to the user. For now, let’s use a default of 3 items and see what the recommender system decides to pick for our customer.

In [35]:
rec_items(11000, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,ProductID,ProductName
0,358,"Mountain-200 Black, 38"
1,467,"Half-Finger Gloves, L"
2,563,"Touring-1000 Yellow, 54"
3,362,"Mountain-200 Black, 46"
4,478,Mountain Bottle Cage
5,228,"Long-Sleeve Logo Jersey, S"
6,361,"Mountain-200 Black, 42"
7,352,"Mountain-200 Silver, 38"
8,465,"Half-Finger Gloves, M"
9,564,"Touring-1000 Yellow, 60"


In [36]:
#we can see that the second customer listed has an ID of 11001. Let’s examine their purchases from the training set.
get_items_purchased(11001, product_train, customers_arr, products_arr, item_lookup)

,ProductID,ProductName
35,350,"Mountain-100 Black, 44"
5432,478,Mountain Bottle Cage
5434,477,Water Bottle - 30 oz.
5475,485,Fender Set - Mountain
5485,604,"Road-750 Black, 44"
5491,225,AWC Logo Cap


These recommendations seem quite good! Remember that the recommendation system has no real understanding of what the items are. All it knows is the purchase history

In [37]:
rec_items(11001, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,ProductID,ProductName
0,538,LL Road Tire
1,355,"Mountain-200 Silver, 42"
2,363,"Mountain-200 Black, 46"
3,480,Patch Kit/8 Patches
4,490,"Short-Sleeve Classic Jersey, L"
5,487,Hydration Pack - 70 oz.
6,222,"Sport-100 Helmet, Blue"
7,360,"Mountain-200 Black, 42"
8,479,Road Bottle Cage
9,489,"Short-Sleeve Classic Jersey, M"


In [38]:
#we can see that the third customer listed has an ID of 11002. Let’s examine their purchases from the training set.
get_items_purchased(11002, product_train, customers_arr, products_arr, item_lookup)

,ProductID,ProductName
1,346,"Mountain-100 Silver, 44"
5454,222,"Sport-100 Helmet, Blue"
5585,359,"Mountain-200 Black, 38"
5829,561,"Touring-1000 Yellow, 46"


These recommendations seem quite good! Remember that the recommendation system has no real understanding of what the items are. All it knows is the purchase history

In [39]:
rec_items(11002, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,ProductID,ProductName
0,361,"Mountain-200 Black, 42"
1,575,"Touring-1000 Blue, 54"
2,356,"Mountain-200 Silver, 46"
3,481,"Racing Socks, M"
4,358,"Mountain-200 Black, 38"
5,576,"Touring-1000 Blue, 60"
6,353,"Mountain-200 Silver, 38"
7,573,"Touring-1000 Blue, 46"
8,578,"Touring-2000 Blue, 50"
9,563,"Touring-1000 Yellow, 54"


These recommendations seem quite good! Remember that the recommendation system has no real understanding of what the items are. All it knows is the purchase history.

In [40]:
#we can see that the third customer listed has an ID of 11003. Let’s examine their purchases from the training set.
get_items_purchased(11003, product_train, customers_arr, products_arr, item_lookup)

,ProductID,ProductName
5432,478,Mountain Bottle Cage
5434,477,Water Bottle - 30 oz.
5457,541,Touring Tire
5458,530,Touring Tire Tube
5515,361,"Mountain-200 Black, 42"
5683,564,"Touring-1000 Yellow, 60"


In [41]:
rec_items(11003, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,ProductID,ProductName
0,487,Hydration Pack - 70 oz.
1,484,Bike Wash - Dissolver
2,354,"Mountain-200 Silver, 42"
3,353,"Mountain-200 Silver, 38"
4,479,Road Bottle Cage
5,362,"Mountain-200 Black, 46"
6,563,"Touring-1000 Yellow, 54"
7,575,"Touring-1000 Blue, 54"
8,573,"Touring-1000 Blue, 46"
9,225,AWC Logo Cap


In [42]:
#we can see that the third customer listed has an ID of 11003. Let’s examine their purchases from the training set.
get_items_purchased(11004, product_train, customers_arr, products_arr, item_lookup)

,ProductID,ProductName
81,345,"Mountain-100 Silver, 42"
5440,214,"Sport-100 Helmet, Red"
5475,485,Fender Set - Mountain
5510,355,"Mountain-200 Silver, 42"
5595,562,"Touring-1000 Yellow, 50"


In [43]:
rec_items(11004, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,ProductID,ProductName
0,363,"Mountain-200 Black, 46"
1,358,"Mountain-200 Black, 38"
2,483,Hitch Rack - 4-Bike
3,576,"Touring-1000 Blue, 60"
4,360,"Mountain-200 Black, 42"
5,314,"Road-150 Red, 56"
6,574,"Touring-1000 Blue, 50"
7,573,"Touring-1000 Blue, 46"
8,465,"Half-Finger Gloves, M"
9,351,"Mountain-100 Black, 48"
